In [3]:
import argparse
import os
import pandas as pd
import boto3
from io import StringIO

In [6]:
def validate_data(bucket, key, endpoint_url, access_key, secret_key):
    print(f"Validation start for s3://{bucket}/{key}")
    s3_client = boto3.client(
        "s3",
        endpoint_url=endpoint_url,
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key
    )
    print(f"erfolgreich")
    
    try:
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        body = obj['Body'].read().decode('utf-8')
        df = pd.read_csv(StringIO(body))

        # --- Basic Validation Logic ---
        print("File loaded successfully.")
        print(f"Shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        expected_cols = ['Datum','RGSCode', 'Geschlecht', 'Verweildauer', 'Altersgruppe']
        if not all(col in df.columns for col in expected_cols):
             raise ValueError(f"Missing expected columns. Found: {df.columns.tolist()}, Expected: {expected_cols}")

        # Example: Check for excessive nulls in target
        if df['target'].isnull().sum() > len(df) * 0.1: # More than 10% nulls
            raise ValueError("Excessive null values found in 'target' column.")

        print("Validation checks passed.")
    except Exception as e:
        print(f"Validation FAILED: {e}")
        raise # Re-raise exception to fail the Argo step

In [5]:
TEST_DF_URL = "/home/amelie/k8s-ml-pipeline/sample_data/Abgang_AL_Geschlecht_Altersgruppen_VWD_RGS.csv"
csv_content = pd.read_csv(TEST_DF_URL, encoding = "latin1", sep=";" )
print(f"pdf gelesen")
csv_content.head(5)

pdf gelesen


,Datum,RGSCode,RGSName,Geschlecht,Verweildauer,Altersgruppe,ABGANG,DS_VWD,Unnamed: 8
0,2019-01-31,101,Eisenstadt,Frauen,0 bis 90 Tage,bis 19 Jahre,4,26,NaN
1,2019-01-31,101,Eisenstadt,Frauen,0 bis 90 Tage,20 bis 24 Jahre,12,39,NaN
2,2019-01-31,101,Eisenstadt,Frauen,0 bis 90 Tage,25 bis 29 Jahre,10,42,NaN
3,2019-01-31,101,Eisenstadt,Frauen,0 bis 90 Tage,30 bis 34 Jahre,16,39,NaN
4,2019-01-31,101,Eisenstadt,Frauen,0 bis 90 Tage,35 bis 39 Jahre,14,41,NaN
